## 对爬取的招聘信息进行数据清洗以及简单的数据分析

In [1]:
# 加载使用到的工具库
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn.ensemble import RandomForestRegressor
from wordcloud import WordCloud
from pylab import mpl
mpl.rcParams["font.sans-serif"] = ["FangSong"] #将图像字体设置为FangSong
mpl.rcParams["axes.unicode_minus"] = False #防止字体显示不出负号
%matplotlib inline

### 数据清洗
<p>将数据或是表头转换为方便阅读的形式，如时间格式，表头的顺序，空白数据。
<p>将无效的数据删除（不含数据分析关键字的数据）。
<p>将重复含义的数据合并
<p>将资格要求拆分
<p>将工资换算为平均工资
<p>将工资为面议的部分用随机森林方法计算出（可能会有较大误差，但我主要看趋势）。

In [2]:
#读取数据、删除重复项目，顺便看看有多少条招聘信息
df = pd.read_excel(r"20180913liepin.xlsx") 
df = df.drop_duplicates() #删除重复值
len(df)

952

In [3]:
df.head(),df.tail()

(           company                                            context  \
 0  法国赛科技术工程集团中国总公司  Responsibilities:•\tTake the lead on the instr...   
 1         广汇汽车华东大区  岗位职责：1、负责从KPI、BI等系统中导出数据对大区运营条线进行数据分析支持；2、收集运营...   
 2       上海谷米实业有限公司  工作职责：一、客户信息维护1、新开客户信息的掌握&熟知二、样机管理（严格遵循样机管理制度）三...   
 3     上海东昌汽车管理有限公司  岗位职责：1、监督并支持集团下属各汽车品牌各售后经理的日常管理工作；2、全国各4S店售后保养...   
 4       上海谷米实业有限公司  职责描述：1、负责大数据需求调研、数据分析、商业分析和数据挖掘模型等，对行业数据进行挖掘分析...   
 
                                               labels           position  \
 0           ['周末双休', '交通便利', '五险一金', '外企环境', '带薪年假']        整车仪表数据分析工程师   
 1  ['绩效奖金', '带薪年假', '交通补助', '通讯津贴', '节日礼物', '管理规范...             数据分析专员   
 2    ['五险一金', '年底双薪', '绩效奖金', '全勤奖', '包吃包住', '带薪年假']  sop专员/销售运营专员/数据分析   
 3  ['带薪年假', '通讯津贴', '午餐补助', '绩效奖金', '定期体检', '年度旅游...            售后数据分析岗   
 4  ['年底双薪', '带薪年假', '绩效奖金', '节日礼物', '五险一金', '岗位晋升...            资深数据分析师   
 
                       qualifications release_date  \
 0   ['学历不限', '经验不限', '语言不限', '年龄不限']  201

In [4]:
#替换一下日期格式
df["release_date"] = df.release_date.apply(lambda t: datetime.strftime(datetime.strptime(t, "%Y年%m月%d日"),"%Y.%m.%d"))

In [5]:
#将空标签替换为"no_labels"
df = df.replace("[]", np.nan)
df.fillna("no_labels", inplace=True)

In [6]:
#拆分资格要求

#由于字符串里面有很多标点符号，为了方便阅读，
#先将'替换成空
df["qualifications"] = df["qualifications"].apply(lambda x:x.replace("'", "")) 

#然后将qualification特征分解为四个特征
for feature,i in {"education":0, "experience":1, "language":2, "age":3}.items():
    df[feature] = df["qualifications"].apply(lambda x: x.lstrip("[").rstrip("]").split(",")[i])

#最后将qualifications特征删除
df.drop(["qualifications"], 1, inplace=True)

In [7]:
#有一些招聘信息并不是我们想要的，我们通过查看职位以及
#职责描述里面是否包含"数据分析"或"data analysis"字样来清
#除一些非数据相关信息

#设置一个新特征来判断是否为数据分析职位，不是的设置为nan
df["isDA"] = df["position"].apply(lambda x: 1 if x.find("数据" or "分析") >= 0  
                                  else 1 if x.find("data" or  "analysis") >= 0 else np.nan)
#df["isDA"] = df["context"].apply(lambda x: 1 if x.find("数据分析") >= 0  
 #                                else 1 if x.find("data analysis") >= 0 else np.nan)

df = df.dropna() #将与数据分析无关的职位删除

df.drop(["isDA"], 1, inplace=True)
len(df)

109

In [ ]:
#看一下age,education,experience中有没有描述意义重复的词
for feature in ["age", "education", "experience", "language"]:
    print("*"*80)
    print(df[feature].unique())

In [ ]:
#统一语言的描述
df = df.replace({' 普通话':' 语言不限',
                 ' 英语 + 普通话':' 英语',
                 ' 英语 + 日语 + 普通话':' 英语 + 日语'})

#将统招本科与本科及以上合并
df = df.replace("统招本科", "本科及以上")

In [ ]:
#将工资转换成平均工资，面议部门暂时保留
df["salary"] = df.salary.apply(lambda x: x[:10].strip())
df["low_salary"] = df["salary"].apply(lambda x:"面议" if x=="面议" else int(x.split("-")[0]))
df["high_salary"] = df["salary"].apply(lambda x:"面议" if x=="面议" else int(x.split("-")[1][:-1]))
df = df.replace("面议", 0)
df["ave_salary"] = (df["low_salary"] + df["high_salary"])/2
df.drop(["salary", "low_salary", "high_salary"], 1, inplace=True)
df = df.replace(0, "面议")

In [ ]:
#用特征学历和工作经验来预测面议工资
#先将学历和工作经验分类数值化
df["experience"] = df["experience"].map({" 经验不限":0, " 1年以上":1, " 2年以上":2, " 3年以上":3, 
                                                 " 4年以上":4, " 5年以上":5, " 6年以上":6, " 7年以上":7, 
                                                 " 8年以上":8, " 10年以上":10, " 12年以上":12})
df["education"] = df["education"].map({"学历不限":0, "大专及以上":1, "本科及以上":2, "硕士及以上":3, "博士":4})

#将有工资的部分作为训练集，面议部分作为测试集
df_train = df[df["ave_salary"] != "面议"]
df_test = df[df["ave_salary"] == "面议"]

#我们只提取出数值部分来建立模型
df_train = df_train[["experience", "education", "ave_salary"]]
df_test = df_test[["experience", "education"]]
X_train = df_train[["experience", "education"]]
y_train = df_train["ave_salary"]

#建立模型并预测
rfr = RandomForestRegressor() #用随机森林模型预测面议的工资
rfr.fit(X_train, y_train) 
y_pred = rfr.predict(df_test)
y_pred = y_pred.round(1) #将工资调整为小数点后1位

In [ ]:
#将面议工资替换成预测的y_pred
df.loc[df["ave_salary"] == "面议", "ave_salary"] = y_pred
df.ave_salary = df.ave_salary.astype("float64")

In [ ]:
#最后将education和experience由数值型转换为文本
df["experience"] = df["experience"].map({0:"经验不限", 1:"1年以上", 2:"2年以上", 3:"3年以上", 
                                                 4:"4年以上", 5:"5年以上", 6:"6年以上", 7:"7年以上", 
                                                 8:"8年以上", 10:"10年以上", 12:"12年以上"})
df["education"] = df["education"].map({0:"学历不限", 1:"大专及以上", 2:"本科及以上", 3:"硕士及以上", 4:"博士"})

In [ ]:
#我们将第一行调整为方便阅读的顺序
col = ['release_date','company',  'zone','position', 
       'ave_salary', 'education', 'experience',
       'language', 'age', 'labels', 'context']
df = df[col]

### 数据分析
<p>对数据的总体信息简单的描述性统计分析
<p>对一些独立值较多的特征制作词云
<p>采用图形法对工资以及相关特征做了简单的分析

In [ ]:
df.describe(include="O")

#### 简单的描述统计分析
<p> 公司信息  
从发布时间可以看到当天发布的信息占到了27%左右；   
    428个公司提供了693条招聘信息；   
    超过六成的招聘信息没有具体说明公司所在位置；   
    虽然搜索的关键字是数据分析，但职名称多达612条独立职位，可能还存在不是我们需要找的职位。
<p> 职位信息  
    本科以上，3年工作经验，年龄语言不限时出现最多的词条；     
    42%的公司没有贴出labels（但不代表没有福利）。   

#### 对数据进行简单的分析
<p> 主要针对工资，教育，工作经验，语言，进行分析，并查看一下公司和福利
<p> 于是我们再删除一些不需要的数据

In [ ]:
#将release_date,zone,age特征暂时删除
df = df.drop(["release_date", "zone", "age"],1)

In [ ]:
#制作词云

#创建一个制作词云的函数
def create_cloud(feature, figsize=(10,10)):
    """
    制作某一文本特征的词云
    feature:dataframe中特征的seires形式
    figsize:词云图形大小
    """
    
    font = r"C:\Windows\Fonts\simhei.ttf" #选择字体
    
    #将feature中的所有文本放入text
    text = "" 
    for data in feature:
        text += data + ","
    
    #创建图云
    fig = plt.figure(figsize=figsize)
    cloud = WordCloud(background_color="white", 
                      font_path=font, 
                      max_font_size=100,
                     stopwords=["no_labels", "上海"]).generate(text)
    plt.axis("off") #隐藏坐标轴
    plt.imshow(cloud, interpolation="bilinear") 
    print(feature.name)
    plt.show()

In [ ]:
#大致看一下公司、职位、福利都有哪些信息
for feature in ["company", "position", "labels"]:
    create_cloud(df[feature])

In [ ]:
import jieba
contexts = ""
for context in df["context"]:
    if not context.isalnum():
        contexts += context 
seg_list = jieba.cut(contexts)
wl = "".join(seg_list)
cloud = WordCloud(background_color="white", 
                    font_path=r"C:\Windows\Fonts\simhei.ttf",
                    max_font_size=100,
                    stopwords=["no_labels", "上海"]).generate(wl)
plt.axis("off") #隐藏坐标轴
plt.imshow(cloud, interpolation="bilinear") 
print("context")

In [ ]:
seg_list

<p> 从company中可以发现基本都是汽车相关公司。
<p> 从position中可以看到manager和Engineer出现频率较高。可能与我们需要找的数据分析职位有出入，因为一些技术类、销售类或是经济类相关职位也会有对数据进行分析的职位描述，而且现在很多企业的高层会通过数据来驱动公司业务，可见，越来越多的职位需要数据分析的能力。  
<p> labels里面五险一金最多，但我认为这条是公司对员工的基本义务，不应该出现在福利里面；发展空间大出现的频率也很高，也不能算是福利，而且带有一种画饼的嫌疑。其他字样也都是些很暧昧的字样，对于职场老鸟来说不具备吸引力。怪不得有297个信息里都没有labels，可能求职者也不太会关注这些labels。

In [ ]:
#来看一下工作经验、学历、语言和工资的关系

#定义一个画图函数
def draw_relating_salary(feature, index):
    """
    画出各个特征关于工资的关系图
    feature: 特征
    index: 横坐标的顺序(由于dataframe的排列顺序并不是我们平时惯用的顺序，故x轴自行设定)
    """
    
    #提取特征中的数据作为x轴，y轴
    feature_count = df.groupby([feature])["ave_salary"].count()
    feature_count = feature_count.reindex(index)
    feature_value = feature_count.values
    
    feature_salary = df.groupby([feature])["ave_salary"].mean()
    feature_salary = feature_salary.reindex(index)
    feature_salary_value = feature_salary.values
    
    #设置双坐标系
    fig, left_ax = plt.subplots(figsize=(10,8)) 
    right_ax = left_ax.twinx()    
    
    #作图
    left_ax.bar(index, feature_value, color="grey", edgecolor="black", alpha=0.5) #条形图各条件出现次数
    right_ax.plot(index, feature_salary_value, "C4o-") # 折线图各条件下的平均工资
    
    #设置标签
    plt.title(feature, size=20) 
    left_ax.set_ylabel("Number",size=17)
    right_ax.set_ylabel("Annual Salary", size=17)
    
    #设置坐标轴字体
    left_ax.tick_params(labelsize=15)
    right_ax.tick_params(labelsize=15)
    left_ax.xaxis.set_tick_params(rotation=30)#横坐标倾斜，防止文本重叠
    
    #给图形添加数据
    for a,b in zip(plt.xticks()[0], feature_value):
        left_ax.text(a, b+2, b, ha="center", size=15)
    for a,b in zip(plt.xticks()[0], feature_salary_value):
        right_ax.text(a, b+1, "%.0f" % b, ha="center", va="bottom", size=15, color="C4")

In [ ]:
#工作经验与工资的关系
x_exp = ['经验不限', '1年以上', '2年以上', '3年以上', '4年以上', '5年以上',
     '6年以上', '7年以上', '8年以上', '10年以上', '12年以上' ] #设置方便阅读的x轴顺序
draw_relating_salary("experience", x_exp)

<p> 俗话说3年、5年是一道坎啊，而我们的数据中3年和5年是出现的频数最多，根据1W小时定理，工作3-5年可以算是这个领域的专家了。
<p> 工资与工作经验基本成正比，虽然12年以上的比10年以上的少了很多，但考虑到爬取的样本很少，可以忽略其参考价值。
<p> 可以看出，即使是在传统汽车行业，2年数据分析的工作经验也能有平均24万年薪，5年的工作经验也能超过30万，但考虑到我们爬取的信息包含有一部分其它职位（只是职责描述中有数据分析的职位），存在一定的不确定因素。

In [ ]:
#学历与工资的关系
x_edu = ['学历不限', '大专及以上', '本科及以上', '硕士及以上', '博士']
draw_relating_salary("education", x_edu)

<p> 学历与工资基本呈正比，博士的工资虽然比硕士少，但博士的样本太少，参考价值不大，况且硕士及以上也包含有博士的意思。
<p> 学历不限与本科及以上工资差别不大，可以猜想有些公司虽然对学历没有要，可能实际上还是以招本科生为基准。
<p> 本科及以上出现的次数远多于其他学历，可以看出该行业对学历的要求不算太严格，本科就行。

In [ ]:
#语言与工资的关系
x_lan = [' 语言不限', ' 英语', ' 日语', ' 英语 + 日语', ' 英语 + 德语']
draw_relating_salary("language", x_lan)

<p>语言不限的频数远多于有语言要求的，说明除了外企，数据分析这一职位对语言要求不高
<p>有语言要求的频数虽然不多，但可以看到包含日语（大概率是日企）企业给出的工资远远低与其他语言要求；作为在日企上班的的我觉得，虽然样本数量少，但还是能说明一些问题的，日企你们给这么低的薪水真的想招人么？

### 存在问题
<p> 虽然最终得到了693条招聘信息，但并没有对context进行深入挖掘，应该还存在一部分无效职位
<p> 我用RandomForestRegression对工资面议的部分进行了预测，但实际上该模型对训练样本的准确率只有4成，但我只需要预测的部分趋势正确即可（主要原因是本人能力有限TOT）。
<p> 由于本人也是新手，应该还存在我也不知道的问题。

In [ ]:
df["position"].values